In [0]:
import numpy as np 
import pandas as pd
import requests
import bs4
import time
from bs4 import BeautifulSoup

# assembling the entire webscraping process
max_results_per_city = 50
# crawling more results will also take much longer
city_set = ['Toulouse']
# creating sample dataframe to scraped data
cols = ['city', 'job_title', 'company_name', 'summary', 'salary']

database = []

for start in range(0, 20000, 10):    
    
    page = requests.get('https://www.indeed.fr/emplois?q=developpeur+-data' + '&start=' + str(start))
    time.sleep(1)
    soup = BeautifulSoup(page.text, "lxml")

    for div in soup.find_all(name="div", attrs={"class":"row"}):
        
        job_post =[]
        c = div.findAll(name='span', attrs={'class': 'location accessible-contrast-color-location'})
        for span in c:
            if span:
                job_post.append(span.text)
            else:
                job_post.append("NAN")
                
        # job title
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            job_post.append(a["title"])

        # company name
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:   
            # if at least one span tag with class : company, contain the company name
            for b in company:
                job_post.append(b.text.strip())
        else: 
            # if not, in a specialty tag of class : result-link-source
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                job_post.append(span.text)

        # summary text
        d = div.findAll('div', attrs={'class': 'summary'})   
        #find all span tags in entry with class : summary
        for span in d:
            job_post.append(span.text.strip())

        # salary data
        d = div.findAll('span', attrs={'class': 'salaryText'})  
        for span in d:
            if span:
                job_post.append(span.text)
            else:
                job_post.append("NAN")  

        database.append(job_post)


df=pd.DataFrame(database, columns=cols)

In [0]:
df.to_json('dev_offer.json')

In [0]:
from google.colab import files
files.download('dev_offer.json') 

In [0]:
df

,city,job_title,company_name,summary,salary
0,Béthune (62),DEVELOPPEUR (H/F),Fidei RH,FIDEI RH recrute pour son client « société de ...,None
1,Limonest (69),WANTED : DEVELOPPEUR/SE PHP EN ALTERNANCE,LDLC,Le groupe LDLC propose un poste de développeur...,None
2,Montrouge (92),Développeur(se) junior H/F,CA CIB FRANCE,Crédit Agricole CIB est la banque de financeme...,None
3,Paris (75),CONCEPTEUR / DEVELOPPEUR INFORMATICA - SI ACTU...,CNP Assurances,CNP Assurances est le premier assureur de pers...,None
4,Lannion (22),Ingénieur Développement Logiciel - Alternance,Ericsson,Vous avez déjà acquis certaines compétences te...,None
...,...,...,...,...,...
20210,Évreux (27),DÉVELOPPEUR COMMERCIAL (H/F),Profil Search,"Profil Search, cabinet de recrutement spéciali...",\n24 000 € - 26 000 € par an
20211,Bordeaux (33),DEVELOPPEUR PHP MAGENTO 2 (H/F) – BORDEAUX – R...,Fortil,"Rejoindre FORTIL Bordeaux, c’est intégrer une ...",None
20212,Sainghin-en-Mélantois (59),Developpeur Outils Pricing H/F,Norauto,"Rejoindre NORAUTO, c'est intégrer l’enseigne l...",None
20213,Paris (75),Développeur Php F/H,Kent FR,Manager et encadrer les développeurs.\nIntéres...,\n40 000 € - 45 000 € par an
